In [1]:
import collections

import pandas as pd

In [2]:
%cd ../..

/Users/mwtmurphy/projects/advent-of-code


/Users/mwtmurphy/Library/Caches/pypoetry/virtualenvs/advent-of-code-rjxGAg1e-py3.10/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
char_map = {"T": 10, "J": 11, "Q": 12, "K": 13, "A": 14}

def map_card(card: str, char_map: dict = char_map) -> int:
    '''Returns integer value of card'''
    
    if card in ["T", "J", "Q", "K", "A"]:
        card_val = char_map[card]
    else:
        card_val = int(card)

    return card_val

In [6]:
with open("data/2023/day_7.txt") as file:
    day_7_lines = file.read().strip().splitlines()


In [7]:
hands = [line.split(" ")[0] for line in day_7_lines]
hand_df = pd.DataFrame([[map_card(card) for card in hand] for hand in hands])

type_rank = []

# group index values by hand type
for ix, hand in hand_df.iterrows():
    hand_counter = collections.Counter(hand)

    if len(hand_counter) == 1:
        type_rank.append(7) # five of a kind

    elif len(hand_counter) == 2:
        if hand_counter.most_common(1)[0][1] == 4:
            type_rank.append(6) # four of a kind
        else:
            type_rank.append(5) # full house
    
    elif len(hand_counter) == 3:
        if hand_counter.most_common(1)[0][1] == 3:
            type_rank.append(4) # three of a kind
        else:
            type_rank.append(3) # two pair
    
    elif len(hand_counter) == 4:
        type_rank.append(2) # one pair
    
    else:
        type_rank.append(1) # high card

hand_df["type"] = type_rank

hand_df["bid"] = [line.split(" ")[1] for line in day_7_lines]
hand_df["bid"] = hand_df["bid"].astype(int)

hand_df = hand_df.sort_values(by=["type", 0, 1, 2, 3, 4], ascending=False).reset_index(drop=True)
hand_df["winnings"] = (hand_df.shape[0] - hand_df.index) * hand_df["bid"]

total_winnings = hand_df["winnings"].sum()
print("day 7a solution:", total_winnings)

day 7a solution: 255048101
